In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

%matplotlib inline

### Load dataset

In [ ]:
data = pd.read_csv(url, names=['sepal.length', 'sepal.width', 'petal.length', 'petal.width', 'variety'],sep=',')
#data = pd.read_excel(file)

print('shape of the input data {}'.format(data.shape))
random_state = 42 #use this to make the experiment repeatable

#some methods for DataFrame 
data.columns #returns columns
data.head(n=5)  #returns first n elements
data.describe() #returns a short summary of descriptive statistics
data['name_of_column'].describe() # descriptive statistics for specific column
data.describe(include= "O") #also includes categorical attributes
sorted(pd.unique(data['target_name'])) #unique takes as input an 1D array like

pd.DataFrame.hist(data, figsize = [10,10])#show histograms. data can be a column
plt.hist(data['sepal.width']) #histogram on single column
plt.show()
sns.pairplot(data, hue = 'variety', diag_kind='kde') #hue=attribute chosen as class


from sklearn.preprocessing import OneHotEncoder
# Merge after changing from nominal to number
ohe = OneHotEncoder(handle_unknown= "ignore", sparse= False)
sex = ohe.fit_transform(df0[["Sex"]])
sex_df = pd.DataFrame(sex, columns= df0["Sex"].unique())
sex_df["Index"] = sex_df.index

df1 = df0.drop("Sex", axis= 1).merge(sex_df, on= "Index").set_index("Index")

from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder(dtype = int)
df[2] = enc.fit_transform(df[2].values.reshape(-1,1))

######## Correlation ########################
corr = data[data.columns].corr()
plt.figure(figsize=(15,10)) # set X and Y size
sns.heatmap(corr, cmap="YlGnBu", annot=True)
#############################################
######## Boxplot ############################
plt.figure(figsize=(15,15))
pos = 1
for i in data.columns[:-1]:
    plt.subplot(2, 2, pos)
    sns.boxplot(x= data['variety'], y = data[i], data = data) #with label and divided by class
    pos += 1
sns.boxplot(data = data) #alternative way (all in one image)

### Tree classifier

### Classification with hyperparameter tuning

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score, train_test_split

target_name = 'quality' # our y
sorted(pd.unique(data[target_name])) #unique takes as input an 1D array like
classes = sorted(data[target_name].unique())
print(classes) 
#X = data.drop(labels='species',axis=1)
#y = data.drop(labels=['sepal length', 'sepal width', 'petal length', 'petal width'],axis=1)
X = data.drop(target_name,axis=1)
y = data[target_name]
Xtrain, Xtest, ytrain, ytest = train_test_split(X,y,train_size=0.66,random_state=random_state)

####### Decision Tree ##########################
model = DecisionTreeClassifier(criterion='entropy',random_state=random_state)
model.fit(Xtrain,ytrain)
predicted_ytrain = model.predict(Xtrain)
training_accuracy = accuracy_score(ytrain,predicted_ytrain)*100
print('Accuracy on training set is {}%'.format(training_accuracy))
predicted_ytest = model.predict(Xtest)
testing_accuracy = accuracy_score(ytest,predicted_ytest)*100
fitted_max_depth = model.tree_.max_depth # _ is used to access attributes in scikit_lear
parameter_values = range(1,fitted_max_depth+1)
print('Accuracy on training set is {}%'.format(testing_accuracy))
print('Maximum depth of the tree is {}'.format(fitted_max_depth))

###### Tuning with cross validation #############
scores = []
for par in parameter_values:
    model = DecisionTreeClassifier(criterion='entropy', random_state=random_state, max_depth=par)
    score = cross_val_score(model, Xtrain, ytrain, scoring = 'accuracy', cv = 5)
    mean_score = np.mean(score)
    scores.append(mean_score)
print(scores)

##### Printing the scores ####################
plt.figure(figsize=(10,5))
plt.plot(parameter_values,scores,'-o',linewidth=2, markersize=14)
plt.xlabel('max depth')
plt.ylabel('accuracy')
plt.title("Score with Cross Validation varying max_depth of tree", fontsize = 15)
plt.show

###### Use the best hyperparameter #########
best_parameter = parameter_values[np.argmax(scores)]
model = DecisionTreeClassifier(criterion='entropy', random_state=random_state, max_depth=best_parameter)
model.fit(Xtrain,ytrain)
testing_accuracy_best_parameter = model.predict(Xtest)
best_accuracy = accuracy_score(ytest,testing_accuracy_best_parameter)*100
print('Accuracy on training set is {:.1f}% with depth {}'.format(best_accuracy,best_parameter))


print(classification_report(ytest,testing_accuracy_best_parameter))

print(confusion_matrix(ytest,testing_accuracy_best_parameter))

print('\t\t\t\tAccuracy \tHyperparameter')
print('Simple Holdout and full tree: \t{:0.1f}\t\t{}'.format(testing_accuracy,fitted_max_depth))
print('CrossValidation and tuning: \t{:0.1f}\t\t{}'.format(best_accuracy,best_parameter))

####### plot Tree #######
plt.figure(figsize=(10,10))
plot_tree(model,filled=True,feature_names=['sepal length', 'sepal width', 'petal length', 'petal width'],\
          class_names=['setosa', 'versicolor', 'virginica'],rounded=True,proportion=True)
plt.show()

maybe add last exercise: change quality to binary value

### Using several classifiers

In [ ]:
import warnings
warnings.filterwarnings('ignore') # uncomment this line to suppress warnings
from sklearn import datasets
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier

print(__doc__) # print information included in the triple quotes at the beginning
dataset = datasets.load_iris()
ts = 0.3
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=ts,random_state=random_state)
model_lbls = [
              'dt', 
              'nb', 
              'lp', 
              'svc', 
             'knn',
             'adb',
             'rf',
            ]

# Set the parameters by cross-validation
tuned_param_dt = [{'max_depth': [*range(1,20)]}]
tuned_param_nb = [{'var_smoothing': [10, 1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-07, 1e-8, 1e-9, 1e-10]}]
tuned_param_lp = [{'early_stopping': [True]}]
tuned_param_svc = [{'kernel': ['rbf'], 
                    'gamma': [1e-3, 1e-4],
                    'C': [1, 10, 100, 1000],
                    },
                    {'kernel': ['linear'],
                     'C': [1, 10, 100, 1000],                     
                    },
                   ]
tuned_param_knn =[{'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}]
tuned_param_adb = [{'n_estimators':[20,30,40,50],
                   'learning_rate':[0.5,0.75,1,1.25,1.5]}]
tuned_param_rf = [{'max_depth': [*range(5,15)],
                   'n_estimators':[*range(10,100,10)]}]

models = {
    'dt': {'name': 'Decision Tree       ',
           'estimator': DecisionTreeClassifier(), 
           'param': tuned_param_dt,
          },
    'nb': {'name': 'Gaussian Naive Bayes',
           'estimator': GaussianNB(),
           'param': tuned_param_nb
          },
    'lp': {'name': 'Linear Perceptron   ',
           'estimator': Perceptron(),
           'param': tuned_param_lp,
          },
    'svc':{'name': 'Support Vector      ',
           'estimator': SVC(), 
           'param': tuned_param_svc
          },
    'knn':{'name': 'K Nearest Neighbor ',
           'estimator': KNeighborsClassifier(),
           'param': tuned_param_knn
       },
       'adb':{'name': 'AdaBoost           ',
           'estimator': AdaBoostClassifier(),
           'param': tuned_param_adb
          },
    'rf': {'name': 'Random forest       ',
           'estimator': RandomForestClassifier(),
           'param': tuned_param_rf
          }
}

scores = ['precision', 'recall']
def print_results(model):
    print("Best parameters set found on train set:")
    print()
    # if best is linear there is no gamma parameter
    print(model.best_params_)
    print()
    print("Grid scores on train set:")
    print()
    means = model.cv_results_['mean_test_score']
    stds = model.cv_results_['std_test_score']
    params = model.cv_results_['params']
    for mean, std, params_tuple in zip(means, stds, params):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params_tuple))
    print()
    print("Detailed classification report for the best parameter set:")
    print()
    print("The model is trained on the full train set.")
    print("The scores are computed on the full test set.")
    print()
    y_true, y_pred = y_test, model.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

results_short = {}
for score in scores:
    print('='*40)
    print("# Tuning hyper-parameters for %s" % score)
    print()

    #'%s_macro' % score ## is a string formatting expression
    # the parameter after % is substituted in the string placeholder %s
    for m in model_lbls:
        print('-'*40)
        print("Trying model {}".format(models[m]['name']))
        
        clf = GridSearchCV(models[m]['estimator'], models[m]['param'], cv=5,scoring='%s_macro' % score,n_jobs = 2) # this allows using multi-cores
        clf.fit(X_train,y_train)
        print_results(clf)
        results_short[m] = clf.best_score_
    print("Summary of results for {}".format(score))
    print("Estimator")
    for m in results_short.keys():
        print("{}\t - score: {:4.2}%".format(models[m]['name'], results_short[m]))

### Linear regression

In [ ]:
import scipy.stats
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

data.shape[0] - data.dropna().shape[0] # number of rows with null
data = data.dropna() #axis 0 (row) by default
######  Data Transformation #####################
data['SexHRP']=data['SexHRP'].apply(lambda x: 0 if (x=='Female') else 1) #change value from Female,Male to 0,1
#data.SexHRP.replace(('Male', 'Female'),(1,0),inplace=True)
data['qmeat_hhsize_ratio'] = data['qmeat']/data['hhsize'] #new column created using 2 previous columns
data['income_hhsize_ratio'] = data['income']/data['hhsize']
data = data[['adults_n', 'children_n', 'SexHRP', 'AgeHRP', 'qmeat_hhsize_ratio', 'income_hhsize_ratio', 'uvmeat']]

target = "qmeat_hhsize_ratio"
y = data[target]
X = data.drop(target, axis=1)

####### two dimensional scatter plots 
##### for all the predicting variables with respect to the target
ncols=3
import math
nrows = math.ceil((data.shape[1]-1)/ncols)
figwidth = ncols * 7
figheigth = nrows*5
fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(figwidth, figheigth),sharey=True)
plt.subplots_adjust(hspace=0.5)
fig.suptitle("Predicting variables versus target", fontsize=18, y=0.95)

for c, ax in zip(X.columns,axs.ravel()):
    data.sort_values(by=c).plot.scatter(x=c,y=target, title = '"{}" versus "{}"'.format(target,c)
                                    , ax=ax)
plt.figure(figsize=(figwidth, figheigth))
plt.suptitle("target versus predicting variables")
for i, col in enumerate(X.columns):
    plt.subplot(nrows, ncols, i + 1)
    plt.title(f"\"{target}\" versus \"{data.columns[i-1]}\"")
    plt.scatter(X[col], y)
    plt.ylabel(target)
    plt.xlabel(col)

###### Show p-values of targets with respect to variables
from sklearn.feature_selection import f_regression
# Your code here
what is the p-value
_, p_values = f_regression(X,y)
p_values_show = pd.DataFrame({'Variable': X.columns, 'p-value': p_values})
p_values_show

###### Split the data
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=random_state, train_size=0.7)
print('Training set and test set have {} and {} elements respectively'.format(X_train.shape[0],X_test.shape[0]))
X_train.shape

##### Consider a reduced dataset containing the chosen variable and the target
pred_var = 'adults_n'
X_train_r = X_train[pred_var].values.reshape(-1,1) # transform a series into a one-column array
X_test_r = X_test[pred_var].values.reshape(-1,1)
X_train_r.shape

#####Fit the `linear_model` estimator on the training set and predict the target for the test set using the *fitted* estimator
# Create linear regression object
linear_uni = linear_model.LinearRegression()
# Train the model using the training set
linear_uni.fit(X_train_r, y_train)
# Make predictions using the test set
y_train_pred_uni = linear_uni.predict(X_train_r)
y_test_pred_uni = linear_uni.predict(X_test_r)

#### f-test #########
def f_test(y_true, y_pred, n_var, n_obs, sn=.95):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    n = n_obs
    p = n_var+1 # number of regression parameters (coefficients + intercept)
    y_true_m = np.mean(y_true)
    SSM = np.sum((y_pred-y_true_m)**2)
    SST = np.sum((y_true-y_true_m)**2)
    SSE = np.sum((y_true-y_pred)**2)
    DFT = n - 1
    DFM = p - 1 # degrees of freedom for model - numerator
    DFE = n - p # degrees of freedom for error - denominator
    DFT = n - 1
    MSM = SSM / DFM
    MSE = SSE / DFE 
    MST = SST / DFT
    F = MSM / MSE
    p = 1-scipy.stats.f.cdf(F, DFM, DFE) #find p-value of F test statistic 
    return F, p

###### Compute the statistical significance of the model
#perform F-test
f_statistic_uni, p_value_uni = f_test(y_train, y_train_pred_uni, X_train_r.shape[1], X_train_r.shape[0])

# The coefficient
coeff_uni = linear_uni.coef_[0] # the coefficient is returned as a one-element list
intercept_uni = linear_uni.intercept_
# The root mean squared error
rmse_uni = mean_squared_error(y_test, y_test_pred_uni, squared=False)
# Coefficient of determination = 1 is perfect prediction
r2_uni = r2_score(y_test, y_test_pred_uni)

# The results are assembled in a dataframe for a compact view
pd.DataFrame({'Univariate Linear - Value' : [intercept_uni
                        , coeff_uni
                        , rmse_uni
                        , r2_uni
                        , f_statistic_uni
                        , p_value_uni]}
            , index = ['Intercept for "{}"'.format(pred_var)
                     , 'Coefficient for "{}"'.format(pred_var)
                     , 'rmse'
                     , 'r2'
                     , 'f-statistic'
                     , 'p-value'])

####### Using the entire dataset ######################
# Create linear regression object
linear_multi = linear_model.LinearRegression()
# Train the model using the training set
linear_multi.fit(X_train, y_train)
# Make predictions using the test set
y_train_pred_multi = linear_multi.predict(X_train)
y_test_pred_multi = linear_multi.predict(X_test)

# Show the coefficients of the predicting variables
pd.DataFrame({'Variable': X.columns, 'Coefficient': linear_multi.coef_})

_, p_values = f_regression(X_train,y_train_pred_multi)
p_values_show = pd.DataFrame({'Variable': X.columns, 'p-value': p_values})
p_values_show

#perform F-test
f_statistic_multi, p_value_multi = f_test(y_train, y_train_pred_multi, X_train.shape[1], X_train.shape[0])
                                        
# The mean squared error
rmse_multi = mean_squared_error(y_test, y_test_pred_multi, squared=False)
# print("The MSE for the multivariate linear regression of '{}' is: {:8.2f}"\
#     .format(target, rmse_dt))
# Coefficient of determination=1 is perfect prediction
r2_multi = r2_score(y_test, y_test_pred_multi)
# print("The 'R square' for the multivariate linear regression of '{}' is: {:8.2f}"\
#     .format(target, r2_dt))
pd.DataFrame({'Multivariate Linear - Value' : [rmse_multi
                        , r2_multi]}
            , index = ['rmse'
                     , 'r2'])

# The results are assembled in a dataframe for a compact view
pd.DataFrame({'Univariate Linear - Value' : [rmse_multi
                        , r2_multi
                        , f_statistic_multi
                        , p_value_multi]}
            , index = ['rmse'
                     , 'r2'
                     , 'f-statistic'
                     , 'p-value']).style.format(precision=4)

####### Decision Tree Multivariate Regression
from sklearn.tree import DecisionTreeRegressor
data = DecisionTreeRegressor(random_state=random_state)
# Train the model using the training set
data.fit(X_train, y_train);
max_max_depth = data.tree_.max_depth
print("The maximum depth of the full Decision Tree Regressor is {}".format(max_max_depth))

from sklearn.model_selection import GridSearchCV
param_grid = {'max_depth': list(range(1,max_max_depth))}
# create the grid search cross validation object
dt_gscv = GridSearchCV(estimator=DecisionTreeRegressor(random_state=random_state)
                    , param_grid=param_grid
                    , scoring='neg_mean_squared_error' # select model with minimum mse
                    )
dt_gscv.fit(X_train,y_train)
dt_best = dt_gscv.best_estimator_ # the GridSearchCV returns the best estimator
best_max_depth = dt_best.tree_.max_depth
print("The optimal maximum depth for the decision tree is {}".format(best_max_depth))

# Make predictions using the test set
y_test_pred_dt = dt_best.predict(X_test)

rmse_dt = mean_squared_error(y_test, y_test_pred_dt, squared=False)

print("Decision Tree Regression - RMSE = {:.2f}".format(rmse_dt))

from sklearn.tree import plot_tree
from matplotlib.pyplot import figure
figure(figsize = (20,15))
plot_tree(dt_best
          , filled = True # fills nodes with colors related to classes darker color means higher purity
          , feature_names = list(X.columns)
          # , max_depth=2
          , fontsize=18
        #   , class_names = df[target].unique()
         )



###### Random Forest Multivariate Regression
# Create Random Forest regression object
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state=random_state)
# for simplicity, we use as a maximum maximum depth of the tree the value found in
# the unconstrained decision tree fitting
param_grid_rf = {'max_depth': list(range(1,max_max_depth))
}
# create the grid search with cross validation
rf_gscv = GridSearchCV(rf, param_grid=param_grid_rf, scoring='neg_mean_squared_error') # look for minimum mean square error
# Train the model using the training set
rf_gscv.fit(X_train, y_train)
# the grid search returns the best estimator
rf = rf_gscv.best_estimator_
print("The optimal maximum depth for the trees in the random forest is {}".format(rf.max_depth))

y_test_pred_rf = rf.predict(X_test)
rmse_rf = mean_squared_error(y_test, y_test_pred_rf, squared=False)
print("Random Forest Regression - RMSE = {:.2f}".format(rmse_rf))

### Polynomial regression

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
#### Skipped until data subdivision into X and y
plt.scatter(X,y)
plt.xlabel("Temperature");
plt.ylabel("Demand");
plt.show()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=random_state, train_size=0.7)
print('Training set and test set have {} and {} elements respectively'.format(X_train.shape[0],X_test.shape[0]))

def f_test(y_true, y_pred, n_var, n_obs, sn=.95):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    n = n_obs
    p = n_var+1 # number of regression parameters (coefficients + intercept)
    y_true_m = np.mean(y_true)
    SSM = np.sum((y_pred-y_true_m)**2)
    SST = np.sum((y_true-y_true_m)**2)
    SSE = np.sum((y_true-y_pred)**2)
    DFT = n - 1
    DFM = p - 1 # degrees of freedom for model - numerator
    DFE = n - p # degrees of freedom for error - denominator
    DFT = n - 1
    MSM = SSM / DFM
    MSE = SSE / DFE 
    MST = SST / DFT
    F = MSM / MSE
    # f = np.var(x, ddof=1)/np.var(y, ddof=1) #calculate F test statistic 
    p = 1-scipy.stats.f.cdf(F, DFM, DFE) #find p-value of F test statistic 
    return F, p

def print_eval(X, y, model):
    pred = model.predict(X)
    F, p = f_test(y, pred, X.shape[1], X.shape[0])
    print(" Mean squared error: \t{:.5}".format(mean_squared_error(y,pred)))
    print(" r2 score: \t\t{:.5}".format(r2_score(y,pred)))
    print(" f-statistic: \t\t{:.5}".format(F))
    print(" p-value: \t\t{:.5}".format(p))
    return mean_squared_error(pred, y), r2_score(pred, y), F, p

####### First Experiment #########################
lmodel = LinearRegression()
lmodel.fit(X.temp.values.reshape(-1,1), y)
lin = print_eval(X_test, y_test, lmodel)
####### Visualize the prediction of the model #######
lpred = lmodel.predict(np.arange(min(X.temp), max(X.temp)).reshape(-1,1))
plt.plot(np.arange(min(X.temp), max(X.temp)),lpred, label = "lin", color = "red")
plt.legend()
plt.scatter(X,y)
plt.show()


####### Second Experiment ##########
polFea = PolynomialFeatures(2,include_bias=False)
X_poly = polFea.fit_transform(X_train.values)#.reshape(-1,1))
model = LinearRegression()
model.fit(X_poly, y_train)

pol = print_eval(polFea.transform(X_test), y_test, model)

pred = model.predict(polFea.transform((np.arange(min(X.temp), max(X.temp))).reshape(-1,1)))
plt.plot(np.arange(min(X.temp), max(X.temp)),pred, label = "poly",color="red")
plt.legend()
plt.scatter(X,y)
plt.plot()


####### Third Experiment ##########
polFea = PolynomialFeatures(3,include_bias=False)
print("Polynomial degree = 3")
X_poly = polFea.fit_transform(X_train.values)#.reshape(-1,1))
model = LinearRegression()
model.fit(X_poly, y_train)

pol3 = print_eval(polFea.transform(X_test), y_test, model)

pred3 = model.predict(polFea.transform((np.arange(min(X.temp), max(X.temp))).reshape(-1,1)))
plt.plot(np.arange(min(X.temp), max(X.temp)),pred3, label = "poly",color="red")
plt.legend()
plt.scatter(X,y)
plt.plot()

####### Fourth Experiment ##########
polFea = PolynomialFeatures(4,include_bias=False)
print("Polynomial degree = 4")
X_poly = polFea.fit_transform(X_train.values)#.reshape(-1,1))
model = LinearRegression()
model.fit(X_poly, y_train)

pol4 = print_eval(polFea.transform(X_test), y_test, model)

pred4 = model.predict(polFea.transform((np.arange(min(X.temp), max(X.temp))).reshape(-1,1)))
plt.plot(np.arange(min(X.temp), max(X.temp)),pred4, label = "poly",color="red")
plt.legend()
plt.scatter(X,y)
plt.plot()

performance = {"linear ": [*lin],
                "polynomial d = 2": [*pol],
                "polynomial d = 3": [*pol3],
                "polynomial d = 4": [*pol4] }
res = pd.DataFrame(performance, index = ['rmse'
                     , 'r2'
                     , 'f-statistic'
                     , 'p-value'])
res

### Clustering (K-mean)

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import ParameterGrid
from math import sqrt
import warnings
warnings.filterwarnings("ignore")


#dataset_copy = dataset.loc["Fresh":"Delicassen"].apply(np.sqrt)
X_sqrt = pd.concat([dataset.iloc[:,:2],dataset.iloc[:,2:].applymap(sqrt)],axis=1)
mms = MinMaxScaler()
X = pd.DataFrame(mms.fit_transform(X_sqrt), columns = X_sqrt.columns)

####### Elbow method ##########
k_range = list(range(2,11)) # set the range of k values to test 
parameters_km = [{'n_clusters': k_range}]
pg = list(ParameterGrid(parameters_km))
inertias_km = []
silhouette_scores_km = []
for i in range(len(pg)):
    km = KMeans(**(pg[i]), random_state=random_state)
    y_km = km.fit_predict(X)
    inertias_km.append(km.inertia_)
    silhouette_scores_km.append(silhouette_score(X,y_km))

########## Plot the inertia and silhouette score
def two_plots(x, y1, y2, xlabel, y1label, y2label):
    fig, ax1 = plt.subplots()

    color = 'tab:red'
    ax1.set_xlabel(xlabel)
    ax1.set_ylabel(y1label, color=color)
    ax1.plot(x, y1, color=color)
    ax1.tick_params(axis='y', labelcolor=color)
    ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

    color = 'tab:blue'
    ax2.set_ylabel(y2label, color=color)  # we already handled the x-label with ax1
    ax2.plot(x, y2, color=color)
    ax2.tick_params(axis='y', labelcolor=color)
    ax2.set_ylim(0,1) # the axis for silhouette is [0,1]

    fig.tight_layout()  # otherwise the right y-label is slightly clipped
    plt.show()

    two_plots(x=k_range, y1=inertias_km, y2=silhouette_scores_km
          , xlabel='Number of clusters', y1label='Inertias', y2label='Silhouette scores'
         )
    
#### Choose max silhouette score and point of biggest change for inertias
k=3
km = KMeans(n_clusters=k, 
            random_state=random_state)
y_km = km.fit_predict(X)
print("Number of clusters = {}\t- Distortion = {:6.2f}\t- Silhouette score = {:4.2f}"\
    .format(k,inertias_km[k_range.index(k)],silhouette_scores_km[k_range.index(k)]))

### Pie graph ############
clust_sizes_km = np.unique(y_km,return_counts=True)
pd.DataFrame(clust_sizes_km[1]).plot.pie(y=0, autopct='%1.1f%%', );
plt.show()

### Agglomerative Clustering

In [ ]:
from sklearn.cluster import AgglomerativeClustering
parameters = [{'n_clusters': k_range
                    , 'linkage' : ['ward', 'complete', 'average', 'single']}]
pg = list(ParameterGrid(parameters))
result_ac = []
for i in range(len(pg)):
    ac = AgglomerativeClustering(**(pg[i]))
    y_ac = ac.fit_predict(X)
    result_ac.append([pg[i]['linkage'],pg[i]['n_clusters'],silhouette_score(X,y_ac)])

df_result_ac = pd.DataFrame(data = result_ac, columns=['linkage','n_clusters','silhouette_score'])
df_result_ac.sort_values(by='silhouette_score', ascending=False).head(5)

from sklearn.preprocessing import OrdinalEncoder
oe = OrdinalEncoder()
df_result_ac["linkage_enc"] = oe.fit_transform(df_result_ac["linkage"].values.reshape(-1, 1))


fig = plt.figure(figsize=(8, 15))
ax = fig.add_subplot(projection= "3d")

x, y, z = df_result_ac['linkage_enc'].values, df_result_ac['n_clusters'].values, df_result_ac['silhouette_score'].values

bottom = np.zeros(df_result_ac.shape[0])
width = .5 * np.ones(df_result_ac.shape[0])#np.ones_like(zpos)
depth = .5 * np.ones(df_result_ac.shape[0])

ax.bar3d(x, y, bottom, width, depth, z )
ax.set_xlabel("linkage type")
ax.set_ylabel("n_clusters")
ax.set_zlabel("silhouette_score")

plt.show()


print(df_result_ac.iloc[[1]])
ac = AgglomerativeClustering(**(pg[1]))
y_ac = ac.fit_predict(X)
##### Pie graph
clust_sizes_ac = np.unique(y_ac,return_counts=True)
pd.DataFrame(clust_sizes_ac[1]).plot.pie(y=0, autopct='%1.1f%%', );
plt.show()


dataset['cluster_km']=y_km
sns.pairplot(data=dataset, hue='cluster_km')
plt.show()

dataset['cluster_ac']=y_ac
sns.pairplot(data=dataset.drop('cluster_km',axis=1), hue='cluster_ac');
plt.show()

from sklearn.metrics import pair_confusion_matrix
pcm = pair_confusion_matrix(y_km,y_ac)
pcm / pcm.sum()
print("The percentage of match between the two clustering schemes is {:6.2f}%"\
    .format((pcm / pcm.sum()).diagonal().sum()*100))

### DBSCAN

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import ParameterGrid

### Inspect data#########
sns.pairplot(pd.DataFrame(X))
plt.show()

X = X[:,[0,1]]
focus = [0,1]

plt.scatter(X[:,focus[0]], X[:,focus[1]]
            , c='white'          # color filling the data markers
            , edgecolors='black' # edge color for data markers
            , marker='o'         # data marker shape, e.g. triangles (v<>^), square (s), star (*), ...
            , s=50)              # data marker size
plt.grid()  # plots a grid on the data
plt.show()

db = DBSCAN()
y_db = db.fit_predict(X)
db.eps,db.min_samples

cluster_labels_all = np.unique(y_db)
cluster_labels = cluster_labels_all[cluster_labels_all != -1]
n_clusters = len(cluster_labels)
if cluster_labels_all[0] == -1:
    noise = True
    print("There is noise")
else:
    noise = False
print("There is/are {} cluster(s)".format(n_clusters-noise))

######## Print Centroids with auxiliary function from other file
cluster_centers = np.empty((n_clusters,X.shape[1]))
for i in cluster_labels:
    cluster_centers[i,:] = np.mean(X[y_db==i,:], axis = 0)
plot_clusters(X,y_db,dim=(focus[0],focus[1]), points = cluster_centers)
###################################################

####### Find best hyperparams #############################
param_grid = {'eps': list(np.arange(0.01, 1, 0.01)), 'min_samples': list(range(1,10,1))}
params = list(ParameterGrid(param_grid))

dbscan_out = pd.DataFrame(columns =  ['eps','min_samples','n_clusters','silhouette', 'unclust%'])
for i in range(len(params)):
    db = DBSCAN(**(params[i]))
    y_db = db.fit_predict(X)
    cluster_labels_all = np.unique(y_db)
    cluster_labels = cluster_labels_all[cluster_labels_all != -1]
    n_clusters = len(cluster_labels)
    if n_clusters > 1:
        X_cl = X[y_db!=-1,:]
        y_db_cl = y_db[y_db!=-1]
        silhouette = silhouette_score(X_cl,y_db_cl)
        uncl_p = (1 - y_db_cl.shape[0]/y_db.shape[0]) * 100
        dbscan_out.loc[len(dbscan_out)] = [db.eps, db.min_samples, n_clusters, silhouette, uncl_p]

sil_thr = 0.7  # visualize results only for combinations with silhouette above the threshold
unc_thr = 10 # visualize results only for combinations with unclustered% below the threshold
n_clu_max_thr = 4
dbscan_out[(dbscan_out['silhouette']>=sil_thr)\
         & (dbscan_out['unclust%']<=unc_thr)\
         & (dbscan_out['n_clusters']<=n_clu_max_thr)]


############ Observe visually ##############
db = DBSCAN(eps=0.99, min_samples=9)
# db = DBSCAN(eps=0.05, min_samples=9)
# db = DBSCAN(eps=0.16, min_samples=9)
y_db = db.fit_predict(X)
cluster_labels_all = np.unique(y_db)
cluster_labels = cluster_labels_all[cluster_labels_all != -1]
n_clusters = len(cluster_labels)

cluster_centers = np.empty((n_clusters,X.shape[1]))
for i in cluster_labels:
    cluster_centers[i,:] = np.mean(X[y_db==i,:], axis = 0)

print("There are {} clusters".format(n_clusters))
print("The cluster labels are {}".format(cluster_labels))
cluster_centers
plot_clusters(X,y_db,dim=(focus[0],focus[1]), points = cluster_centers)


silhouette = silhouette_samples(X,y_db)
# from plot_silhouette import plot_silhouette  # python script provided separately
from plot_silhouette_w_mean import plot_silhouette  # python script provided separately
plot_silhouette(silhouette,y_db)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np

def plot_clusters(X, y, dim, points,
                  labels_prefix = 'cluster', 
                  points_name = 'centroids',
                  colors = cm.tab10, # a qualitative map 
                      # https://matplotlib.org/examples/color/colormaps_reference.html
#                   colors = ['brown', 'orange', 'olive', 
#                             'green', 'cyan', 'blue', 
#                             'purple', 'pink'],
#                   points_color = 'red'
                  points_color = cm.tab10(10) # by default the last of the map (to be improved)
                 ):
    """
    Plot a two dimensional projection of an array of labelled points
    X:      array with at least two columns
    y:      vector of labels, length as number of rows in X
    dim:    the two columns to project, inside range of X columns, e.g. (0,1)
    points: additional points to plot as 'stars'
    labels_prefix: prefix to the labels for the legend ['cluster']
    points_name:   legend name for the additional points ['centroids']
    colors: a color map
    points_color: the color for the points
    """
    # plot the labelled (colored) dataset and the points
    labels = np.unique(y)
    for i in range(len(labels)):
        color = colors(i / len(labels)) # choose a color from the map
        plt.scatter(X[y==labels[i],dim[0]], 
                    X[y==labels[i],dim[1]], 
                    s=10, 
                    c = [color], # scatter requires a sequence of colors
                    marker='s', 
                    label=labels_prefix+str(labels[i]))
    plt.scatter(points[:,dim[0]], 
                points[:,dim[1]], 
                s=50, 
                marker='*', 
                c=[points_color], 
                label=points_name)
    plt.legend()
    plt.grid()
    plt.show()   

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

def plot_silhouette(silhouette_vals, y, 
 					colors = cm.tab10
					):
    """
    Plotting silhouette scores for the individual samples of a labelled data set.
    The scores will be grouped according to labels and sorted in descending order.
    The bars are proportional to the score and the color is determined by the label.
    
    silhouette_vals: the silhouette values of the samples
    y:               the labels of the samples
    
    """
    cluster_labels = np.unique(y)
    n_clusters = len(cluster_labels)
    y_ax_lower, y_ax_upper = 0, 0
    yticks = []
    for i, c in enumerate(cluster_labels): # generate pairs index, cluster_label
        c_silhouette_vals = silhouette_vals[y==c] # extracts records with the current cluster label
        c_silhouette_vals.sort() # sort the silhouette vals for the current class
        y_ax_upper += len(c_silhouette_vals)
        color = colors(i / n_clusters)
        plt.barh(range(y_ax_lower, y_ax_upper), c_silhouette_vals, height=1.0, 
                edgecolor='none', color=color)
        yticks.append((y_ax_lower + y_ax_upper) / 2)
        y_ax_lower += len(c_silhouette_vals)

    silhouette_avg = np.mean(silhouette_vals)
    plt.axvline(silhouette_avg, color="red", linestyle="--") 
    plt.yticks(yticks, cluster_labels)# + 1)
    plt.ylabel('Cluster')
    plt.xlabel('Silhouette coefficient')
    plt.tight_layout()
    # plt.savefig('./figures/silhouette.png', dpi=300)
    plt.show()

In [ ]:
def plot_silhouette(silhouette_vals, y, 
 					colors = cm.tab10,
 					plot_noise = False
					):
    """
    Plotting silhouette scores for the individual samples of a labelled data set.
    The scores will be grouped according to labels and sorted in descending order.
    The bars are proportional to the score and the color is determined by the label.
    
    silhouette_vals: the silhouette values of the samples
    y:               the labels of the samples
    plot_noise:      boolean, assumes the noise to be labeled with a negative integer
    
    """
    cluster_labels = np.unique(y)
    if not plot_noise:
	    cluster_labels = cluster_labels[cluster_labels != -1]
    n_clusters = len(cluster_labels)
    y_ax_lower, y_ax_upper = 0, 0
    yticks = []
    for i, c in enumerate(cluster_labels): # generate pairs index, cluster_label
        c_silhouette_vals = silhouette_vals[y==c] # extracts records with the current cluster label
        c_silhouette_vals.sort() # sort the silhouette vals for the current class
        y_ax_upper += len(c_silhouette_vals)
        color = colors(i / n_clusters)
        plt.barh(range(y_ax_lower, y_ax_upper), c_silhouette_vals, height=1.0, 
                edgecolor='none', color=color)
        yticks.append((y_ax_lower + y_ax_upper) / 2)
        c_silhouette_avg = np.mean(c_silhouette_vals)
        plt.axvline(c_silhouette_avg
         			, ymin = y_ax_lower/len(silhouette_vals)
         			, ymax = y_ax_upper/len(silhouette_vals)
        			, color=color, linestyle="-."
        			) 
        y_ax_lower += len(c_silhouette_vals)


    silhouette_avg = np.mean(silhouette_vals)
    plt.axvline(silhouette_avg, color="black", linestyle="--") 
    plt.yticks(yticks, cluster_labels)
    plt.ylabel('Cluster')
    plt.xlabel('Silhouette coefficient - Cluster means: -. Global mean: --')
    plt.tight_layout()
    # plt.savefig('./figures/silhouette.png', dpi=300)
    plt.show()